# 测试代码可行性

In [ ]:
import os

In [5]:
print(os.path.split("D://myCode/PYTHON_file/Surf/测试网络请求的图片识别")[0])
print(os.path.split("")[0])

D://myCode/PYTHON_file/Surf



In [2]:
import base64

def str2base64str(sentence):
    return base64.b64encode(sentence.encode('utf8')).decode('utf8')

def base64str2str(b64str):
    return base64.b64decode(b64str).decode('utf8')

print(base64str2str("QmlnRm9sZGVyXFNtYWxsRm9sZGVy"))

BigFolder\SmallFolder


消息队列

In [2]:
from redis import StrictRedis


conn=StrictRedis("127.0.0.1",6379,db=2,decode_responses=True)


In [23]:
# 添加消息
for i in range(10):
    stream_id=conn.xadd("mq_group",{"name":"mrli","flag":i})

In [24]:
# 查询长度
conn.xlen("mq_group")

10

In [21]:
# 删除整个消费者组
conn.delete("mq_group")

1

In [6]:
# 查询消息队列
allmsgs=conn.xrange("nameDB","-","+")
print(allmsgs)

[]


In [18]:
# 删除一个消息（通过id），（不是读取！）
conn.xdel("mq_group","1670830836229-0")

1

In [5]:
# 删除一个消息队列
conn.delete('nameDB')

1

In [25]:
# 创建消费者组（好像用不到
conn.xgroup_create("mq_group","consumergroupA")

True

In [50]:
# 消息队列读
msg=conn.xread({"mq_group":"1670830836229-1"},count=1)
print(msg)

[['mq_group', [('1670831456053-0', {'name': 'mrli', 'flag': '0'})]]]


In [ ]:
from flask_restful import Resource, reqparse
from flask import current_app,abort,Response,jsonify 
import os
from utils.gitmanage import clone,log
from utils.filemanage import foldertree
import time

class PushQueue(Resource):
    
    @staticmethod
    def get():
        """
        :param databse_position:
        """
        parse = reqparse.RequestParser()
        parse.add_argument('databse_position', type=str, help='must need a databse file position', required=True, trim=True)
        parse.add_argument('databse_name', type=str, help='must need a databse name', required=True, trim=True)
        
        args = parse.parse_args()
        file_pos = args['databse_position']
        data_name = args['databse_name']
        now = int(round(time.time()*1000))
        nowstr = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(now/1000))
        # MQ push
        conn.xadd(data_name,{"file_position":file_pos,"time":nowstr})
        return Response("OK",status=200)